Classification models 

In [2]:
import pandas as pd

Combining the Padel Descriptor output for the DNA gyrase inhibitor and decoy molecules

In [3]:
df1 =pd.read_csv(r"C:\Users\saman\Desktop\Gyrase\smiles\smiles_padel.csv")
df2= pd.read_csv(r"C:\Users\saman\Desktop\Gyrase\decoys\decoy_padel.csv")
df= pd.concat([df1, df2], ignore_index=True)
df

,name,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,AUTOGEN_smiles_1,1,-2.1766,4.737588,66.8370,53.407653,0,0,47,26,...,7.662735,53.423678,2.054757,19.603698,9.973243,7.028501,1484,51,1.832,146
1,AUTOGEN_smiles_2,1,-1.3013,1.693382,59.4132,53.407653,6,6,47,26,...,7.662735,53.423678,2.054757,19.603698,9.973243,7.028501,1484,51,1.809,146
2,AUTOGEN_smiles_3,1,-1.0135,1.027182,52.4994,48.185274,6,6,42,24,...,7.884124,49.016084,2.042337,19.968336,7.434296,9.945672,1234,43,1.267,134
3,AUTOGEN_smiles_4,1,-0.0321,0.001030,59.1725,47.965688,6,6,42,26,...,8.740818,52.412431,2.015863,25.240393,7.433893,10.157549,1537,48,2.134,138
4,AUTOGEN_smiles_5,1,-0.0321,0.001030,59.1725,47.965688,6,6,42,26,...,8.740818,52.412431,2.015863,25.240393,7.433893,10.157549,1537,48,2.134,138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,ZINC000000001534,0,-1.9487,3.797432,32.2822,52.309825,12,12,46,21,...,6.113082,43.233659,2.058746,6.890752,0.000000,6.890752,981,25,5.714,104
948,ZINC000000001536,0,1.1368,1.292314,34.9328,45.286274,12,13,38,20,...,7.003439,40.076299,2.003815,5.116616,5.116616,0.000000,828,30,5.024,98
949,ZINC000000001553,0,-0.6576,0.432438,26.0410,36.083516,11,12,30,18,...,8.003370,37.127417,2.062634,14.996255,2.524096,12.472159,660,24,1.973,94
950,ZINC000000001557,0,-0.3141,0.098659,38.3032,52.445032,12,12,46,22,...,6.438888,45.041651,2.047348,9.157403,2.359417,6.797986,1007,36,4.563,116


Assigning labels to the dataframe:  ZINC = decoy(0)
AUTOGEN_ = active(1)

In [4]:
df['Label'] = df['name'].apply(lambda x: 0 if 'ZINC' in x else 1)
df.to_csv("labeled_descriptor.csv", index=False)

C:\Users\saman\AppData\Local\Temp\ipykernel_17544\4236325685.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Label'] = df['name'].apply(lambda x: 0 if 'ZINC' in x else 1)


Data Cleaning and Preprocessing

In [5]:
df_clean= df.drop(columns=["name"])

In [6]:
df_clean =df_clean.dropna(axis=1)

In [7]:
#remove low variance features
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.01)
X = df_clean.drop(columns=["Label"])
y = df_clean["Label"]

X_lowvar = selector.fit_transform(X)

In [8]:
#remove highly correlated features
import numpy as np

# Create a correlation matrix
corr_matrix = pd.DataFrame(X_lowvar).corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Drop features with correlation > 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
X_filtered = pd.DataFrame(X_lowvar).drop(columns=to_drop, axis=1)

In [9]:
#scale the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_filtered)

In [10]:
#Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

Training Multiple Classifiers

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Initialize models
rf = RandomForestClassifier(n_estimators=100, random_state=42)
svm = SVC(kernel='rbf', probability=True, random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb = XGBClassifier(eval_metric='logloss', random_state=42)

models = {
    "Random Forest": rf,
    "Support Vector Machine": svm,
    "XGBoost": xgb
}


In [12]:
#Fit and evaluate models
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    print(f"--- {name} ---")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}\n")


--- Random Forest ---
Confusion Matrix:
[[ 80   1]
 [  6 104]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96        81
           1       0.99      0.95      0.97       110

    accuracy                           0.96       191
   macro avg       0.96      0.97      0.96       191
weighted avg       0.96      0.96      0.96       191

ROC AUC Score: 0.9965

--- Support Vector Machine ---
Confusion Matrix:
[[ 81   0]
 [  6 104]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        81
           1       1.00      0.95      0.97       110

    accuracy                           0.97       191
   macro avg       0.97      0.97      0.97       191
weighted avg       0.97      0.97      0.97       191

ROC AUC Score: 0.9955

--- XGBoost ---
Confusion Matrix:
[[ 80   1]
 [  4 106]]
Classification Report:
              precision    recall  f1-s

Model | Accuracy | ROC AUC | Notes
1) Random Forest | 97% | 0.9993 | Near perfect
2) SVM (RBF kernel) | 97% | 0.9972 | Excellent
3) XGBoost | 100% | 1.0000 | Perfect fit 

Cross Validating with ROC AUC tracking

In [13]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, roc_auc_score

# Create Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Custom scorer for ROC AUC
roc_auc = make_scorer(roc_auc_score, needs_proba=True)

models = {
    "Random Forest": rf,
    "SVM": svm,
    "XGBoost": xgb
}

for name, model in models.items():
    scores = cross_val_score(model, X_scaled, y, cv=skf, scoring=roc_auc)
    print(f"{name} AUC (5-Fold CV): {scores.mean():.4f} ± {scores.std():.4f}")


c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\utils\_param_validation.py", line 194, in wrapper
    params = func_sig.bind(*args, **kwargs)
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\inspect.py", line 3045, in bind
    return self._bind(args, kwargs)
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\inspect.py", line 3034, in _bind
    raise TypeError(
TypeError: got an unexpected

Random Forest AUC (5-Fold CV): nan ± nan


c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\utils\_param_validation.py", line 194, in wrapper
    params = func_sig.bind(*args, **kwargs)
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\inspect.py", line 3045, in bind
    return self._bind(args, kwargs)
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\inspect.py", line 3034, in _bind
    raise TypeError(
TypeError: got an unexpected

SVM AUC (5-Fold CV): nan ± nan


c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\utils\_param_validation.py", line 194, in wrapper
    params = func_sig.bind(*args, **kwargs)
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\inspect.py", line 3045, in bind
    return self._bind(args, kwargs)
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\inspect.py", line 3034, in _bind
    raise TypeError(
TypeError: got an unexpected

XGBoost AUC (5-Fold CV): nan ± nan


c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\site-packages\sklearn\utils\_param_validation.py", line 194, in wrapper
    params = func_sig.bind(*args, **kwargs)
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\inspect.py", line 3045, in bind
    return self._bind(args, kwargs)
  File "c:\Users\saman\Miniconda3\envs\bioenv\lib\inspect.py", line 3034, in _bind
    raise TypeError(
TypeError: got an unexpected

In [14]:
# For AUC scoring (requires predict_proba)
scoring = 'roc_auc'  # for binary classification
from sklearn.svm import SVC
svm = SVC(probability=True)  # This enables predict_proba

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
scores = cross_val_score(model, X, y, cv=5, scoring='roc_auc')
print(f"Random Forest AUC (5-Fold CV): {scores.mean():.2f} ± {scores.std():.2f}")

Random Forest AUC (5-Fold CV): 0.99 ± 0.01


In [16]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy="stratified")
dummy.fit(X_train, y_train)
roc_auc_score(y_test, dummy.predict_proba(X_test)[:, 1])  # Should be ~0.5

0.5267676767676768

Since our DummyClassifier indicates the model is only performing slightly better than random guessing. The score is very close to 0.5 ehich indicates random guessing 

In [18]:
# Check for duplicate columns
duplicate_columns = df.columns[df.columns.duplicated()]

# Check for duplicate rows
duplicate_rows = df[df.duplicated()]

print("Duplicate columns:", duplicate_columns)
print("Number of duplicate rows:", len(duplicate_rows))


Duplicate columns: Index([], dtype='object')
Number of duplicate rows: 0


In [19]:
# Calculate the correlation matrix
correlation_matrix = df.select_dtypes(include=['float64', 'int64']).corr()

# Set a threshold for high correlation (e.g., 0.9)
threshold = 0.9
high_corr_vars = [column for column in correlation_matrix.columns if any(correlation_matrix[column] > threshold)]

# Drop highly correlated columns
df_reduced = df.drop(columns=high_corr_vars)

print(f"Dropped columns with high correlation: {high_corr_vars}")


Dropped columns with high correlation: ['nAcid', 'ALogP', 'ALogp2', 'AMR', 'apol', 'naAromAtom', 'nAromBond', 'nAtom', 'nHeavyAtom', 'nH', 'nC', 'nN', 'nO', 'nS', 'nP', 'nF', 'nCl', 'nBr', 'nI', 'nX', 'ATS0m', 'ATS1m', 'ATS2m', 'ATS3m', 'ATS4m', 'ATS5m', 'ATS6m', 'ATS7m', 'ATS8m', 'ATS0v', 'ATS1v', 'ATS2v', 'ATS3v', 'ATS4v', 'ATS5v', 'ATS6v', 'ATS7v', 'ATS8v', 'ATS0e', 'ATS1e', 'ATS2e', 'ATS3e', 'ATS4e', 'ATS5e', 'ATS6e', 'ATS7e', 'ATS8e', 'ATS0p', 'ATS1p', 'ATS2p', 'ATS3p', 'ATS4p', 'ATS5p', 'ATS6p', 'ATS7p', 'ATS8p', 'ATS0i', 'ATS1i', 'ATS2i', 'ATS3i', 'ATS4i', 'ATS5i', 'ATS6i', 'ATS7i', 'ATS8i', 'ATS0s', 'ATS1s', 'ATS2s', 'ATS3s', 'ATS4s', 'ATS5s', 'ATS6s', 'ATS7s', 'ATS8s', 'AATS0m', 'AATS1m', 'AATS2m', 'AATS3m', 'AATS4m', 'AATS5m', 'AATS6m', 'AATS7m', 'AATS8m', 'AATS0v', 'AATS1v', 'AATS2v', 'AATS3v', 'AATS4v', 'AATS5v', 'AATS6v', 'AATS7v', 'AATS8v', 'AATS0e', 'AATS1e', 'AATS2e', 'AATS3e', 'AATS4e', 'AATS5e', 'AATS6e', 'AATS7e', 'AATS8e', 'AATS0p', 'AATS1p', 'AATS2p', 'AATS3p', 'AA

In [20]:
df_reduced

,name,nB,nBondsQ,nHmisc,nsLi,nssBe,nssssBem,nsBH2,nssBH,nsssB,...,nF4Ring,nF5Ring,n9HeteroRing,n10HeteroRing,n11HeteroRing,nG12HeteroRing,nFHeteroRing,nF4HeteroRing,nF5HeteroRing,nTHeteroRing
0,AUTOGEN_smiles_1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AUTOGEN_smiles_2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AUTOGEN_smiles_3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AUTOGEN_smiles_4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AUTOGEN_smiles_5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,ZINC000000001534,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
948,ZINC000000001536,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
949,ZINC000000001553,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
950,ZINC000000001557,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
label_col = 'Label'  # change this if your label column has a different name


# Separate features and label
X = df.drop(columns=[label_col])
y = df[label_col]

# Keep only numeric columns for correlation filtering
X_numeric = X.select_dtypes(include=[np.number])

# Correlation matrix
corr_matrix = X_numeric.corr().abs()
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Drop highly correlated features (correlation > 0.95)
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.95)]
print(f"Dropping {len(to_drop)} highly correlated columns.")

# Drop correlated features from the original X (to preserve any remaining non-numeric features, if needed)
X_filtered = X.drop(columns=to_drop)

# Recombine with label column
data_cleaned = pd.concat([X_filtered, y], axis=1)


Dropping 572 highly correlated columns.


In [26]:
data_cleaned
data_cleaned.to_csv("filtered_data.csv", index=False)

In [27]:
from sklearn.model_selection import train_test_split

X = data_cleaned.drop(columns=[label_col])
y = data_cleaned[label_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [30]:
# Drop non-numeric columns (except the label)
non_numeric_cols = data_cleaned.select_dtypes(exclude=[np.number]).columns.tolist()
non_numeric_cols = [col for col in non_numeric_cols if col != label_col]

# Drop those columns
data_cleaned = data_cleaned.drop(columns=non_numeric_cols)


In [31]:
# Define features and labels again
X = data_cleaned.drop(columns=[label_col])
y = data_cleaned[label_col]


In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=y)


In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

# Evaluation
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


Confusion Matrix:
 [[ 81   0]
 [  6 104]]

Classification Report:
               precision    recall  f1-score   support

           0       0.93      1.00      0.96        81
           1       1.00      0.95      0.97       110

    accuracy                           0.97       191
   macro avg       0.97      0.97      0.97       191
weighted avg       0.97      0.97      0.97       191

Accuracy: 0.9685863874345549


In [35]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)


In [37]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')  # Suppress convergence warnings for clean output

# Step 1: Impute missing values
imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Step 2: Define models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42)
}

# Step 3: Train and evaluate each model
for name, model in models.items():
    print(f"\n=== {name} ===")
    model.fit(X_train_imputed, y_train)
    y_pred = model.predict(X_test_imputed)

    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))



=== Random Forest ===
Confusion Matrix:
 [[ 80   1]
 [  6 104]]

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.99      0.96        81
           1       0.99      0.95      0.97       110

    accuracy                           0.96       191
   macro avg       0.96      0.97      0.96       191
weighted avg       0.96      0.96      0.96       191

Accuracy: 0.9633507853403142

=== SVM ===
Confusion Matrix:
 [[  4  77]
 [  0 110]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.05      0.09        81
           1       0.59      1.00      0.74       110

    accuracy                           0.60       191
   macro avg       0.79      0.52      0.42       191
weighted avg       0.76      0.60      0.47       191

Accuracy: 0.5968586387434555

=== XGBoost ===
Confusion Matrix:
 [[ 80   1]
 [  3 107]]

Classification Report:
               precision    recall 

Here’s a table summarizing the results for your markdown:

| **Model**            | **Accuracy**  | **Precision (Class 0)** | **Precision (Class 1)** | **Recall (Class 0)** | **Recall (Class 1)** | **F1-Score (Class 0)** | **F1-Score (Class 1)** | **Macro Avg F1-Score** | **Weighted Avg F1-Score** | **Confusion Matrix**              |
|----------------------|---------------|-------------------------|-------------------------|----------------------|----------------------|------------------------|------------------------|------------------------|---------------------------|-----------------------------------|
| **Random Forest**     | 96.3%         | 0.93                    | 0.99                    | 0.99                 | 0.95                 | 0.96                   | 0.97                   | 0.96                   | 0.96                      | [[80, 1], [6, 104]]               |
| **Support Vector Machine** | 59.7%         | 1.00                    | 0.59                    | 0.05                 | 1.00                 | 0.09                   | 0.74                   | 0.42                   | 0.47                      | [[4, 77], [0, 110]]               |
| **XGBoost**           | 97.9%         | 0.96                    | 0.99                    | 0.99                 | 0.97                 | 0.98                   | 0.98                   | 0.98                   | 0.98                      | [[80, 1], [3, 107]]               |
| **Logistic Regression** | 94.8%         | 0.93                    | 0.96                    | 0.95                 | 0.95                 | 0.94                   | 0.95                   | 0.95                   | 0.95                      | [[77, 4], [6, 104]]               |



Hybrid Model (Random Forest + XGBoost) for enhanced prediction using stacking, where the predcitions are combined using a logistic regression model as a meta-classifier

In [43]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

df_cleaned = pd.read_csv("filtered_data.csv")
X_data = data_cleaned.drop(columns=[label_col])
y_data = data_cleaned[label_col]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)

# Initialize individual classifiers
rf = RandomForestClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Initialize the meta-classifier (Logistic Regression in this case)
meta_classifier = LogisticRegression(max_iter=1000)

# Create the hybrid model using stacking
stacking_model = StackingClassifier(
    estimators=[('rf', rf), ('xgb', xgb)],
    final_estimator=meta_classifier
)

# Train the hybrid model
stacking_model.fit(X_train, y_train)

# Make predictions
y_pred = stacking_model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.972027972027972
Confusion Matrix:
[[115   4]
 [  4 163]]
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       119
           1       0.98      0.98      0.98       167

    accuracy                           0.97       286
   macro avg       0.97      0.97      0.97       286
weighted avg       0.97      0.97      0.97       286



Here's a table summarizing all the results for you to add to your markdown:

| Model                | Accuracy  | Precision (Class 0) | Precision (Class 1) | Recall (Class 0) | Recall (Class 1) | F1-score (Class 0) | F1-score (Class 1) | Macro Average | Weighted Average |
|----------------------|-----------|---------------------|---------------------|------------------|------------------|-------------------|-------------------|----------------|------------------|
| **Hybrid Model**      | 97.2%     | 0.97                | 0.98                | 0.97             | 0.98             | 0.97              | 0.98              | 0.97           | 0.97             |
| **XGBoost**           | 97.9%     | 0.96                | 0.99                | 0.99             | 0.97             | 0.98              | 0.98              | 0.98           | 0.98             |
| **Random Forest**     | 96.3%     | 0.93                | 0.99                | 0.99             | 0.95             | 0.96              | 0.97              | 0.96           | 0.96             |
| **Logistic Regression** | 94.8%    | 0.93                | 0.96                | 0.95             | 0.95             | 0.94              | 0.95              | 0.95           | 0.95             |
| **SVM**               | 59.7%     | 1.00                | 0.59                | 0.05             | 1.00             | 0.09              | 0.74              | 0.79           | 0.60             |



For AI-driven drug disovery precision and recall of Class 1 is crucial. Based on our model performance we shall select **XGBoost** as our strongest choice. 